In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## detecting and cropping faces

In [3]:
done = False
done = True

from gaze import init, Data
args = init(batch_size=8, no_cuda=True, verbose=1)
d = Data(args)

Found 1358 sample images;  1087  to train 271 to test


In [4]:
import os
import glob
datapath_faces = 'dataset_faces'
datapath = 'dataset'

if True:
    n_show = 10
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png')[-n_show:]:
            print(filename)

dataset/blink/2018-09-12_Laurent_035.png
dataset/blink/2018-09-12_Laurent_223.png
dataset/blink/2018-09-12_Laurent_237.png
dataset/blink/2018-09-12_Laurent_222.png
dataset/blink/2018-09-14_Laurent_121.png
dataset/blink/2018-09-04_Laurent_180.png
dataset/blink/2018-10-03_Laurent_116.png
dataset/blink/2018-09-12_Laurent_020.png
dataset/blink/2018-09-05_Laurent_085.png
dataset/blink/2018-09-14_Laurent_069.png
dataset/center/2018-09-04_Laurent_16.png
dataset/center/2018-09-12_Laurent_140.png
dataset/center/2018-09-14_Laurent_243.png
dataset/center/2018-09-12_Laurent_168.png
dataset/center/2018-09-12_Laurent_183.png
dataset/center/2018-09-04_Laurent_221.png
dataset/center/2018-09-04_Laurent_235.png
dataset/center/2018-10-03_Laurent_089.png
dataset/center/2018-10-03_Laurent_076.png
dataset/center/2018-09-14_Laurent_096.png
dataset/left/2018-09-12_Laurent_236.png
dataset/left/2018-09-05_Laurent_091.png
dataset/left/2018-09-12_Laurent_008.png
dataset/left/2018-09-05_Laurent_052.png
dataset/lef

In [5]:
import imageio
if not done:
    import time
    from gaze import FaceExtractor
    FE = FaceExtractor()
    timings = []
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
            frame = imageio.imread(filename)
            t0 = time.time()
            face = FE.face_extractor(frame)
            t1 = time.time()
            timings.append(t1-t0)
            if False:
                print("Number of faces detected: {}".format(len(dets)))
                for i, d in enumerate(dets):
                    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                        i, d.left(), d.top(), d.right(), d.bottom()))

            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the image
                ax.imshow(frame)

                # Create a Rectangle patch
                rect = patches.Rectangle((d.bottom(), d.left()), d.right()-d.left(), d.top()-d.bottom(), linewidth=1, edgecolor='r', facecolor='none')

                # Add the patch to the Axes
                ax.add_patch(rect)
                plt.show()    
            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the cropped image
                ax.imshow(face)

                plt.show()                

            filename_face = filename.replace(datapath, datapath_faces)
            imageio.imwrite(filename_face, face) 


In [6]:
if not done:
    timings = np.array(timings) * 1000
    print('timings in ms =', timings.mean(), '+/-', timings.std()) 

In [7]:
if not done:
    fig, ax = plt.subplots(figsize=((8, 5)))
    n, bins, patches = ax.hist(timings, bins=np.linspace(100, 200, 100), alpha=.4)
    ax.vlines(np.median(timings), 0, n.max(), 'g', linestyles='dashed', label='median = %.3f ms' % np.median(timings))
    #ax.vlines(25, 0, n.max(), 'r', linestyles='dashed', label='chance level')
    #ax.vlines(100, 0, n.max(), 'k', label='max')
    ax.set_xlabel('Timings (ms)')
    ax.set_ylabel('Smarts')
    ax.legend(loc='best')
    if False:
        plt.show() 
    else:
        plt.savefig('dlib_timings.png')

## training the model on the cropped faces

In [8]:
from gaze import init
#epochs = 400
#args = init(verbose=0, epochs=epochs)
args = init(verbose=1)
print('Parameters=', args)

path = '_Regard.pt'

Parameters= {'dataset_folder': 'dataset', 'dataset_faces_folder': 'dataset_faces', 'batch_size': 8, 'test_batch_size': 1, 'size_test_set': 0.2, 'epochs': 20, 'do_adam': False, 'lr': 0.025, 'momentum': 0.05, 'no_cuda': False, 'num_processes': 1, 'seed': 42, 'log_interval': 1, 'fullsize': 64, 'crop': 64, 'size': 64, 'mean': 0.6, 'std': 0.3, 'conv1_dim': 9, 'conv1_kernel_size': 18, 'conv2_dim': 36, 'conv2_kernel_size': 14, 'stride1': 2, 'stride2': 4, 'N_cv': 20, 'dimension': 30, 'verbose': 1}


In [16]:
from gaze import init
epochs = 400
args = init(verbose=0, epochs=epochs)
path = '_Regard_400.pt'

In [17]:
!ls -l {path}
#!rm {path}

ls: _Regard_400.pt: No such file or directory


In [18]:
from gaze import ML
ml = ML(args)
ml.train(path=path)

Training model...
Model saved at _Regard_400.pt


## testing the model

In [19]:
Accuracy = ml.test()
print('Accuracy={:.1f}%'.format(Accuracy*100))


Test set: Average loss: 0.1979, Accuracy: 261/271 (96%)

Accuracy=96.3%


Showing the one which are wrong:

In [20]:
max(ml.dataset.test_loader.dataset.indices)

tensor(1348)

In [21]:
ml.dataset.dataset.imgs[0]

('dataset_faces/blink/2018-09-04_Laurent_100.png', 0)

In [22]:
for idx, (data, target) in enumerate(ml.dataset.test_loader):
    print(target, ml.dataset.dataset.imgs[ml.dataset.test_loader.dataset.indices[idx]])

tensor([1]) ('dataset_faces/center/2018-09-04_Laurent_99.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-05_Laurent_014.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-05_Laurent_007.png', 1)
tensor([0]) ('dataset_faces/blink/2018-09-04_Laurent_172.png', 0)
tensor([2]) ('dataset_faces/left/2018-09-05_Laurent_223.png', 2)
tensor([3]) ('dataset_faces/right/2018-10-03_Laurent_207.png', 3)
tensor([0]) ('dataset_faces/blink/2018-09-05_Laurent_160.png', 0)
tensor([1]) ('dataset_faces/center/2018-09-04_Laurent_208.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-12_Laurent_109.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-05_Laurent_138.png', 1)
tensor([0]) ('dataset_faces/blink/2018-09-05_Laurent_173.png', 0)
tensor([2]) ('dataset_faces/left/2018-10-03_Laurent_056.png', 2)
tensor([1]) ('dataset_faces/center/2018-09-12_Laurent_171.png', 1)
tensor([2]) ('dataset_faces/left/2018-09-12_Laurent_058.png', 2)
tensor([3]) ('dataset_faces/right/2018-09-05_Laurent_128.png', 3)
tensor(

tensor([3]) ('dataset_faces/right/2018-10-03_Laurent_004.png', 3)
tensor([2]) ('dataset_faces/left/2018-09-05_Laurent_083.png', 2)
tensor([0]) ('dataset_faces/blink/2018-09-12_Laurent_132.png', 0)
tensor([0]) ('dataset_faces/blink/2018-09-04_Laurent_44.png', 0)
tensor([2]) ('dataset_faces/left/2018-09-04_Laurent_19.png', 2)
tensor([2]) ('dataset_faces/left/2018-09-14_Laurent_065.png', 2)
tensor([1]) ('dataset_faces/center/2018-09-12_Laurent_003.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-05_Laurent_084.png', 1)
tensor([2]) ('dataset_faces/left/2018-09-05_Laurent_180.png', 2)
tensor([0]) ('dataset_faces/blink/2018-10-03_Laurent_013.png', 0)
tensor([3]) ('dataset_faces/right/2018-09-05_Laurent_077.png', 3)
tensor([0]) ('dataset_faces/blink/2018-10-03_Laurent_040.png', 0)
tensor([1]) ('dataset_faces/center/2018-09-04_Laurent_84.png', 1)
tensor([1]) ('dataset_faces/center/2018-09-12_Laurent_226.png', 1)
tensor([3]) ('dataset_faces/right/2018-09-12_Laurent_127.png', 3)
tensor([2]) (

In [23]:
for idx, (data, target) in enumerate(ml.dataset.dataset):
    print(target, ml.dataset.dataset.imgs[ml.dataset.dataset.indices[idx]])

AttributeError: 'ImageFolder' object has no attribute 'indices'

In [24]:
for data, labels in ml.dataset.test_loader:
    fig, ax = ml.show(only_wrong=True)
    plt.show() 


In [25]:
for data, labels in ml.dataset.dataset:
    fig, ax = ml.show(only_wrong=True)
    plt.show() 
